In [1]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
# Load the IMDB dataset
max_features = 5000
maxlen = 100
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)


In [3]:
# Preprocess the data
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

In [4]:
# Convert sequences back into text
reverse_word_index = dict([(value, key) for (key, value) in imdb.get_word_index().items()])
def decode_review(text):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in text])

X_train_text = [decode_review(x) for x in X_train]
X_test_text = [decode_review(x) for x in X_test]

# Create and fit tokenizer
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train_text)

# Preprocess the data
X_train = pad_sequences(tokenizer.texts_to_sequences(X_train_text), maxlen=maxlen)
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test_text), maxlen=maxlen)

In [16]:
# Build the model
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
# model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))


In [17]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [18]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=32)

Epoch 1/5
782/782 [==============================] - 116s 142ms/step - loss: 0.4117 - accuracy: 0.8087 - val_loss: 0.3419 - val_accuracy: 0.8456
Epoch 2/5
782/782 [==============================] - 105s 134ms/step - loss: 0.2911 - accuracy: 0.8790 - val_loss: 0.3510 - val_accuracy: 0.8427
Epoch 3/5
782/782 [==============================] - 103s 132ms/step - loss: 0.2336 - accuracy: 0.9053 - val_loss: 0.3779 - val_accuracy: 0.8407
Epoch 4/5
782/782 [==============================] - 105s 134ms/step - loss: 0.1944 - accuracy: 0.9220 - val_loss: 0.3942 - val_accuracy: 0.8366
Epoch 5/5
782/782 [==============================] - 105s 134ms/step - loss: 0.1545 - accuracy: 0.9405 - val_loss: 0.4345 - val_accuracy: 0.8423


In [19]:
score, acc = model.evaluate(X_test, y_test, batch_size=32)
print("Test accuracy:", acc)

782/782 [==============================] - 15s 19ms/step - loss: 0.4345 - accuracy: 0.8423
Test accuracy: 0.8422799706459045


In [10]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
def preprocess_input(user_input):
    sequences = tokenizer.texts_to_sequences(user_input)
    return pad_sequences(sequences, maxlen=maxlen)

In [11]:
# Function to predict sentiment
def predict_sentiment(review):
    review = preprocess_input([review])
    prediction = model.predict(review)[0][0]
    if prediction >= 0.5:
        return "Positive"
    else:
        return "Negative"

In [14]:

user_review = input("Enter your movie review (or type 'quit' to exit): ")
sentiment = predict_sentiment(user_review)
print("Predicted sentiment:", sentiment)

Enter your movie review (or type 'quit' to exit): not satisfied by performance
1/1 [==============================] - 0s 45ms/step
Predicted sentiment: Negative
